In [1]:
! sudo cp /home/jovyan/work/jar/spark-cassandra-connector-assembly-3.1.0-11-g53f24ce9.jar /usr/local/spark/jars/spark-cassandra-connector-assembly-3.1.0-11-g53f24ce9.jar

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
#     .config("spark.sql.extensions","com.datastax.spark.connector.CassandraSparkExtensions") \

spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
    .config("spark.cassandra.connection.host", "cassandra0") \
    .config("spark.sql.catalog.cassandra", "com.datastax.spark.connector.datasource.CassandraCatalog") \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

21/10/28 16:51:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


RUN THIS CSQL Code in CQLSH
```
CREATE KEYSPACE test WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1 };
CREATE TABLE test.words (word text , count int, primary key (word) );
INSERT INTO test.words(word, count) VALUES ('and', 50);
```

In [7]:
# read table
spark.read.format("org.apache.spark.sql.cassandra")\
    .options(table="words", keyspace="demo")\
    .load().show()

+----+-----+
|word|count|
+----+-----+
| and|   50|
|  or|   53|
+----+-----+



In [48]:
# another way to read a table
spark.read.table("cassandra.test.words").show()

+----+-----+
|word|count|
+----+-----+
| and|   50|
+----+-----+



In [49]:
# Spark doesn't care about cassandra constraints.
spark.read.table("cassandra.demo.system_utilization").filter("cpu_pct = 5").show()

+--------+-------------------+-------+-------+
|hostname|        measured_on|cpu_pct|     os|
+--------+-------------------+-------+-------+
|    mars|2018-07-19 09:00:00|      5|windows|
|  Saturn|2018-07-19 10:00:00|      5|windows|
|   venus|2018-07-19 09:00:00|      5|    osx|
+--------+-------------------+-------+-------+



In [8]:
# read csv files....
df = spark.read.csv("/home/jovyan/datasets/ufo-sightings/ufo-sightings-2016-*.csv",header=True, inferSchema=True).cache()
df.filter("State is not null").show(10)
df.printSchema()

+-------------+--------------------+-----+---------+-------------+--------------------+--------+
|  Date / Time|                City|State|    Shape|     Duration|             Summary|  Posted|
+-------------+--------------------+-----+---------+-------------+--------------------+--------+
|2/29/16 23:45|        Harbor Beach|   MI|    Light|     1 minute|Yellow/white ball...|3/4/2016|
|2/29/16 23:30|           Sebastian|   FL| Triangle|20-40 minutes|6 low flying craf...|3/4/2016|
|2/29/16 23:00|Salunga/Landisvil...|   PA| Triangle| 5-15 minutes|Pennsylvania tria...|3/4/2016|
|2/29/16 22:00|                York|   PA| Triangle|   30 minutes|Myself and 2 frie...|3/4/2016|
|2/29/16 21:35|              Joliet|   IL|  Unknown|   10 minutes|At approximately ...|3/4/2016|
|2/29/16 20:45|  Port Hope (Canada)|   ON|    Light|   20 minutes|Strange glowing l...|3/4/2016|
|2/29/16 20:29|       Warner Robins|   GA| Fireball|    2 minutes|There were severa...|3/4/2016|
|2/29/16 19:50|              A

# CQL - not an ideal key selection, but this is just a demo
```
create table ufos ( 
    "State" text, 
    "City" text, 
    "Duration" text, 
    "Shape" text, 
    "Summary" text, 
    primary key ( "State", "City")
);
```

In [26]:
# Save To Cassandra!!!

# Make sure there are not nulls in the PK values!
df2 = df.select("City","State","Duration","Shape", "Summary").filter("State is not null").filter("City is not null")

# Write to the cassandra table!
df2.write.format("org.apache.spark.sql.cassandra")\
    .mode('overwrite')\
    .option("confirm.truncate","true")\
    .options(table="ufos", keyspace="demo")\
    .save()

In [27]:
# read back from Cassandra !!!
cdf = spark.read.table("cassandra.demo.ufos")
cdf.show(100)

+-----+--------------------+---------------+---------+--------------------+
|State|                City|       Duration|    Shape|             Summary|
+-----+--------------------+---------------+---------+--------------------+
|   SK|Phalaborwa (South...|      5 minutes|     Cone|Strange light ove...|
|   NC|               Arden|      5 minutes| Triangle|I witnessed one l...|
|   NC|           Asheville|     90 seconds|  Unknown|UFO followed by JETS|
|   NC|              Biscoe|        5 hours|    Light|I have seen this ...|
|   NC|            Calabash|      3 seconds|    Flash|Bright, silvery o...|
|   NC|             Candler|     10 minutes|   Sphere|Cluster of black ...|
|   NC|      Carolina Beach|           5:00|    Other|Reddish-orange li...|
|   NC|           Charlotte|   6-10 seconds|   Circle|Blue orb with aur...|
|   NC|Charlotte (outsid...|     5 minutes+|    Other|Daytime sighting ...|
|   NC|         Cherryville|     30 minutes|  Unknown|Loud horn-like no...|
|   NC|     

In [6]:
df=spark.read.json("/home/jovyan/datasets/weather/weather.json").cache()
df.toPandas().sample(10)
#df.printSchema()

,2020census,city,condition,date,description,dew_point,latitude,longitude,moon_phase,pct_clouds,...,temperature.eve,temperature.max,temperature.min,temperature.morn,temperature.night,timezone,uv_index,wind.direction_deg,wind.gust,wind.speed
129,873965,San Francisco,Rain,2021-10-20,moderate rain,48.83,37.779026,-122.419906,0.50,100,...,57.45,59.50,56.46,57.51,59.50,America/Los_Angeles,0.41,166,30.67,15.97
717,249422,Chesapeake,Clouds,2021-10-24,scattered clouds,52.12,36.718371,-76.246680,0.62,42,...,69.06,72.93,60.12,60.64,62.62,America/New_York,5.00,80,11.25,7.23
860,211328,Rochester,Rain,2021-10-23,light rain,37.80,43.157285,-77.615214,0.59,100,...,48.94,48.94,44.06,45.28,44.06,America/New_York,1.88,176,5.14,4.70
351,442241,Miami,Rain,2021-10-26,light rain,67.30,25.774173,-80.193620,0.68,37,...,80.01,81.97,79.03,79.45,79.66,America/New_York,4.00,63,21.54,19.71
1255,169450,Palmdale,Clear,2021-10-26,clear sky,35.91,34.579313,-118.117111,0.69,1,...,57.97,58.23,46.47,46.47,49.84,America/Los_Angeles,5.00,257,11.01,7.74
1094,187593,Shreveport,Clouds,2021-10-25,broken clouds,67.96,32.522183,-93.765194,0.65,54,...,81.55,87.10,70.68,70.68,78.69,America/Chicago,6.00,172,25.37,8.81
1536,141290,Olathe,Clear,2021-10-19,clear sky,49.23,38.883886,-94.818870,0.47,1,...,67.19,73.54,54.52,55.02,65.08,America/Chicago,4.04,185,39.08,18.54
1300,159467,Alexandria,Clouds,2021-10-23,broken clouds,46.63,38.805110,-77.047023,0.59,78,...,66.83,67.80,55.08,58.23,55.08,America/New_York,2.79,301,21.70,9.15
629,270871,Toledo,Clouds,2021-10-24,broken clouds,29.53,41.652914,-83.537817,0.62,80,...,49.39,50.52,39.45,39.45,49.37,America/New_York,2.00,97,19.57,10.69
1129,181099,Chattanooga,Clouds,2021-10-20,few clouds,51.33,35.045722,-85.309488,0.50,18,...,63.28,74.07,48.96,49.26,59.13,America/New_York,4.64,201,8.21,4.97


In [8]:
df.createOrReplaceTempView("weather")

In [10]:
spark.sql('''
select count(*) from weather
''').show()

+--------+
|count(1)|
+--------+
|    1600|
+--------+



In [14]:
spark.sql('''
select count(*) from (
    select distinct city, state, date from weather
)
''').show()

+--------+
|count(1)|
+--------+
|    1600|
+--------+



In [16]:
df1 = df.toDF("census", "city", "condition", "date", "description",\
              "dew_point", "latitude", "longitude", "moon_phase", "pct_clouds",\
              "pct_humidity", "pressure", "rainfall", "snowfall", "state",\
              "temperature_day", "temperature_eve", "temperature_max",\
              "temperature_min", "temperature_morn", "temperature_night",\
              "timezone", "uv_index", "wind_direction_deg", "wind_gust", "wind_speed")

In [17]:
df1.write.format("org.apache.spark.sql.cassandra")\
    .mode('overwrite')\
    .option("confirm.truncate","true")\
    .options(table="daily_city_weather", keyspace="demo")\
    .save()


In [20]:
weather = spark.read.format("org.apache.spark.sql.cassandra")\
    .options(table="daily_city_weather", keyspace="demo")\
    .load()
weather.filter("condition='Rain'").show()

+-----------+--------------+----------+------+---------+--------------------+---------+--------+---------+----------+----------+------------+--------+--------+--------+---------------+---------------+---------------+---------------+----------------+-----------------+-------------------+--------+------------------+---------+----------+
|       city|         state|      date|census|condition|         description|dew_point|latitude|longitude|moon_phase|pct_clouds|pct_humidity|pressure|rainfall|snowfall|temperature_day|temperature_eve|temperature_max|temperature_min|temperature_morn|temperature_night|           timezone|uv_index|wind_direction_deg|wind_gust|wind_speed|
+-----------+--------------+----------+------+---------+--------------------+---------+--------+---------+----------+----------+------------+--------+--------+--------+---------------+---------------+---------------+---------------+----------------+-----------------+-------------------+--------+------------------+---------+-